In [1]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution3D, ZeroPadding3D, MaxPooling3D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

def create_model():


    print("Creating model.")
    # sys.stdout. Flush()
    nClasses=2
    model = Sequential()
    #model.add(ZeroPadding3d((0, 1, 1), input_shape  = (3, 320, 320, 1)))
    model.add(Convolution3D(32, (3, 3, 3), activation = 'relu', input_shape = (3, 320, 320, 1)))
    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3D(64, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), strides=None, padding='valid'))



    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3d(128, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3D(128, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), strides=None, padding='valid'))

    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3D(256, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3D(256, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3D(256, (3, 3, 3) activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), strides=None, padding='valid'))

    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3d(512, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3d(512, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3d(512, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), strides=None, padding='valid'))

    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3d(512, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3d(512, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((0, 1, 1)))
    model.add(Convolution3d(512, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), strides=None, padding='valid'))

    model.add(Flatten())
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nClasses, activation= 'softmax'))
    # print (model. summary())

    print("Done!")

    #sys.stdout.lush()


IndentationError: unexpected indent (1298847189.py, line 20)

In [ ]:
data_generator = ImageDataGenerator(
            rotation_range=0,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
)
#flow from the directory or flow
